In [6]:
# Script to VNA trace vs power of SMB MW source
#                            -- Dorian
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
from qcodes.loops import Loop
from time import sleep
from tqdm import tqdm

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json

import sys
import os


# IMPORTING THE INSTRUMENTS

In [7]:

sys.path.append('C:\QCodes_Driver_And_Scripts\Drivers')

from RS_SMB_100A import SMB100A
from RS_SGS_100A import SGS100A
from HP3245A import HP3245A
from RS_FSQ import RS_FSQ
from VNA_RS import VNA


try:
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
               init_s_params=False,terminator='\n')

except KeyError as er:


    VNA.close_all() #Disconnect and irreversibly tear down the instrument
    VNA = VNA('ZNB20', 'TCPIP0::192.168.10.2::inst0::INSTR',
               init_s_params=False,terminator='\n')
try:

    MW_source=SMB100A('mw_pump','TCPIP0::192.168.10.6::inst0::INSTR')

except KeyError as er:

    SMB100A.close_all()
    MW_source=SMB100A('mw_pump','TCPIP0::192.168.10.6::inst0::INSTR')



Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062101578, firmware:2.80) in 0.02s
Connected to: Rohde-Schwarz ZNB20-2Port (serial:1311601062101578, firmware:2.80) in 0.02s
Connected to: Rohde&Schwarz SMB100A (serial:1406.6000k03/177959, firmware:3.1.18.2-3.01.203.32) in 0.02s


In [8]:
from qcodes.instrument_drivers.yokogawa.GS200 import GS200

In [9]:
try:
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")


except KeyError as er:

    GS200.close_all() #Disconnect and irreversibly tear down the instrument
    I_source = GS200("gs200", address='TCPIP::192.168.10.3::INSTR', terminator="\n")

Connected to: YOKOGAWA GS210 (serial:90Y438440, firmware:2.02) in 0.03s


# FUNCTION USED TO RUN THE MEASUREMENT

In [10]:
def measure_single_trace(exp,station):

    VNA.rf_on()

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(VNA.channels.S21.trace_mag_phase)

    with meas.run() as datasaver:

        id=datasaver.dataset.run_id
        qc.load_by_run_spec( captured_run_id=id).add_metadata('parameter_snap',
                             json.dumps(parameter_snap))

        get_v = VNA.channels.S21.trace_mag_phase.get()
        datasaver.add_result((VNA.channels.S21.trace_mag_phase, get_v))

# INITIALIZE QCODES EXPERIMENT

In [11]:
start_all_logging()

# Create a station
station = qc.Station()
station.add_component(VNA)
station.add_component(I_source)
station.add_component(MW_source)

station.snapshot()
station.components

# Experiment details
user='DF'
date=datetime.datetime.today().strftime('%Y_%m_%d')
description='VNA_single_trace'
database_name = date+"_"+user+"_"+description


Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\220613-2600-qcodes.log


2022-06-13 16:26:17,920 ¦ qcodes.instrument.base ¦ WARNING ¦ base ¦ snapshot_base ¦ 269 ¦ [gs200_program(GS200Program)] Snapshot: Could not update parameter: trigger


# DATA FOLDER CREATION

In [12]:
script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

# CREATE OR INITIALIZE DATABASE

In [13]:
qc.initialise_or_create_database_at(data_dir+'\\'+database_name+'.db')
qc.config.core.db_location

parameter_snap={}

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.05it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 937.69it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1003.18it/s]


# INSTRUMENTS PARAMETERS

In [14]:
current = -0.
# Snapshot parameters
parameter_snap['flux']={'current': current}

In [15]:
center_freq  = 3.8625
span_freq    = 0.5
start_freq   = center_freq - 0.5*span_freq
end_freq     = center_freq + 0.5*span_freq
# start_freq = 2.
# end_freq   = 12.
step_freq    = 1*1.e-3
VNA_power    = -4
IF_bandwidth =  300
Averages     =  1
points_VNA   = int(abs(end_freq - start_freq)/step_freq)+1
# points_VNA = 2001
trace_time=points_VNA / IF_bandwidth * Averages

print('\ntime for one trace : {} s'.format(datetime.timedelta(seconds = trace_time)))

# Snapshot parameters
parameter_snap['vna'] = {'start_freq':start_freq, 'end_freq':end_freq,
                         'VNA_power':VNA_power, 'IF_bandwidth':IF_bandwidth,
                         'Averages':Averages, 'points':points_VNA}


time for one trace : 0:00:01.670000 s


In [16]:
src_frequency  =  8.92695                #GHz
src_power      = -45.                    #dBm
src_status     = 'off'

# Snapshot parameters
parameter_snap['MW_source'] = {'freq':src_frequency,'power':src_power,'status':src_status}

# SETTING UP INSTRUMENTS

In [17]:
if I_source.output()=='on':
    I_source.output('off')
    
I_source.source_mode('CURR')
I_source.range(10e-3)

I_source.current(current*1.e-3)

I_source.output('on')

In [18]:
VNA.add_channel('S21',vna_parameter='S21')

VNA.channels.S21.format('dB')
VNA.channels.S21.bandwidth(IF_bandwidth)
VNA.channels.S21.power(VNA_power)
VNA.channels.S21.avg(Averages)
VNA.channels.npts(points_VNA)
VNA.channels.start(start_freq*1e9)
VNA.channels.stop(end_freq*1e9)

2022-06-13 16:26:18,210 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\base.py:130: QCoDeSDeprecationWarning: Parameter trace_mag_phase on instrument ZNB20_S21 does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(

2022-06-13 16:26:18,215 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\base.py:130: QCoDeSDeprecationWarning: Parameter trace on instrument ZNB20_S21 does not correctly pass kwargs to its baseclass. A Parameter class must take `**kwargs` and forward them to its baseclass.
  warnings.warn(

2022-06-13 16:26:18,219 ¦ py.warnings ¦ WARNING ¦ warnings ¦ _showwarnmsg ¦ 109 ¦ C:\Users\nicolas.ROCH\Miniconda3\envs\qcodes\lib\site-packages\qcodes\instrument\base.py:130: QCoDeSDeprecationWarning: Parameter trace_fixed_

In [19]:
MW_source.frequency(src_frequency*1e9)
MW_source.power(src_power)
MW_source.status(src_status)

#  MEASUREMENT

In [23]:
# Load experiment
exp_name = 'VNA_single_trace'

sample_name = 'B5_G3_GALV_SQ200_1ST'

exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample_name)

measure_single_trace(exp=exp, station=station)



Starting experimental run with id: 2. 


In [24]:
VNA.rf_off()
VNA.cont_meas_off()
MW_source.status('off')
I_source.output('off')